<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ML-Pipeline-Preparation" data-toc-modified-id="ML-Pipeline-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>ML Pipeline Preparation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#1.-Import-libraries-and-load-data-from-database." data-toc-modified-id="1.-Import-libraries-and-load-data-from-database.-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>1. Import libraries and load data from database.</a></span></li><li><span><a href="#2.-Write-a-tokenization-function-to-process-your-text-data" data-toc-modified-id="2.-Write-a-tokenization-function-to-process-your-text-data-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>2. Write a tokenization function to process your text data</a></span></li><li><span><a href="#3.-Build-a-machine-learning-pipeline" data-toc-modified-id="3.-Build-a-machine-learning-pipeline-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>3. Build a machine learning pipeline</a></span></li><li><span><a href="#4.-Train-pipeline" data-toc-modified-id="4.-Train-pipeline-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>4. Train pipeline</a></span></li><li><span><a href="#5.-Test-your-model" data-toc-modified-id="5.-Test-your-model-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>5. Test your model</a></span></li><li><span><a href="#6.-Improve-your-model" data-toc-modified-id="6.-Improve-your-model-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>6. Improve your model</a></span></li><li><span><a href="#7.-Test-your-model" data-toc-modified-id="7.-Test-your-model-1.0.7"><span class="toc-item-num">1.0.7&nbsp;&nbsp;</span>7. Test your model</a></span></li><li><span><a href="#8.-Try-improving-your-model-further.-Here-are-a-few-ideas:" data-toc-modified-id="8.-Try-improving-your-model-further.-Here-are-a-few-ideas:-1.0.8"><span class="toc-item-num">1.0.8&nbsp;&nbsp;</span>8. Try improving your model further. Here are a few ideas:</a></span></li><li><span><a href="#9.-Export-your-model-as-a-pickle-file" data-toc-modified-id="9.-Export-your-model-as-a-pickle-file-1.0.9"><span class="toc-item-num">1.0.9&nbsp;&nbsp;</span>9. Export your model as a pickle file</a></span></li><li><span><a href="#10.-Use-this-notebook-to-complete-train.py" data-toc-modified-id="10.-Use-this-notebook-to-complete-train.py-1.0.10"><span class="toc-item-num">1.0.10&nbsp;&nbsp;</span>10. Use this notebook to complete <code>train.py</code></a></span></li></ul></li></ul></li></ul></div>

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
import itertools

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import gensim
import gensim.downloader
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table(con=engine,table_name="disaster_tweet")
X = df['message']
Y = df.iloc[:,3:].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Remove non alphanumeric char
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    clean_tokens = [w for w in clean_tokens if not w.lower() in stop_words]
    return clean_tokens

def get_tag(text):
    tokens = tokenize(text)
    return [token for _, token in nltk.pos_tag(tokens)]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [449]:
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer(tokenizer=tokenize,ngram_range=(1, 2))),
    ('tfidf_transformer', TfidfTransformer()),
    ('classifier',MultiOutputClassifier(RandomForestClassifier(n_estimators=20,min_samples_split=10))),    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [450]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [384]:
output_category = list(df.iloc[:,3:].columns)
Y_pred_train = pipeline.predict(X_train)
for i in range(Y_train.shape[1]):
    print(output_category[i])
    print(classification_report(Y_train.T[i], Y_pred_train.T[i]))

related
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      4577
           1       0.99      1.00      0.99     15083

    accuracy                           0.99     19660
   macro avg       0.99      0.97      0.98     19660
weighted avg       0.99      0.99      0.99     19660

request
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     16276
           1       0.99      0.95      0.97      3384

    accuracy                           0.99     19660
   macro avg       0.99      0.97      0.98     19660
weighted avg       0.99      0.99      0.99     19660

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19575
           1       1.00      0.51      0.67        85

    accuracy                           1.00     19660
   macro avg       1.00      0.75      0.84     19660
weighted avg       1.00      1.00      1.00     1966

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17830
           1       1.00      0.96      0.98      1830

    accuracy                           1.00     19660
   macro avg       1.00      0.98      0.99     19660
weighted avg       1.00      1.00      1.00     19660

fire
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19454
           1       1.00      0.82      0.90       206

    accuracy                           1.00     19660
   macro avg       1.00      0.91      0.95     19660
weighted avg       1.00      1.00      1.00     19660

earthquake
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17821
           1       0.99      0.96      0.97      1839

    accuracy                           1.00     19660
   macro avg       0.99      0.98      0.99     19660
weighted avg       1.00      1.00      1.00     19660

col

In [385]:
output_category = list(df.iloc[:,3:].columns)
for i in range(Y_test.shape[1]):
    print(output_category[i])
    print(classification_report(Y_test.T[i], Y_pred.T[i]))

related
              precision    recall  f1-score   support

           0       0.64      0.42      0.51      1544
           1       0.84      0.93      0.88      5010

    accuracy                           0.81      6554
   macro avg       0.74      0.68      0.70      6554
weighted avg       0.79      0.81      0.79      6554

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5464
           1       0.80      0.48      0.60      1090

    accuracy                           0.89      6554
   macro avg       0.85      0.73      0.77      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

              precision    recall  f1-score   support

           0       0.86      0.96      0.91      4674
           1       0.86      0.62      0.72      1880

    accuracy                           0.86      6554
   macro avg       0.86      0.79      0.81      6554
weighted avg       0.86      0.86      0.86      6554

floods
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5983
           1       0.87      0.35      0.50       571

    accuracy                           0.94      6554
   macro avg       0.90      0.67      0.73      6554
weighted avg       0.93      0.94      0.93      6554

storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5941
           1       0.80      0.36      0.50       613

    accuracy                           0.93      6554
   macro avg       0.87      0.68      0.73      6554
weighted avg       0.92      0.93      0.92      6554

fire
 

### 6. Improve your model
Use grid search to find better parameters. 

In [379]:
parameters = {
    'count_vectorizer__ngram_range': ((1, 1), (1, 2), (1, 3)),
    'classifier__estimator__n_estimators': [2,5,10,20],
    'classifier__estimator__min_samples_split': [2,5,10,20,30]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [380]:
cv.fit(X_train,Y_train)

GridSearchCV(estimator=Pipeline(steps=[('count_vectorizer',
                                        CountVectorizer(ngram_range=(1, 2),
                                                        tokenizer=<function tokenize at 0x000002E066457C70>)),
                                       ('tfidf_transformer',
                                        TfidfTransformer()),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=10,
                                                                                               n_estimators=2)))]),
             param_grid={'classifier__estimator__min_samples_split': [2, 5, 10,
                                                                      20, 30],
                         'classifier__estimator__n_estimators': [2, 5, 10, 20],
                         'count_vectorizer__ngram_range': ((1, 1), (1, 2))})

In [381]:
cv.best_params_

{'classifier__estimator__min_samples_split': 10,
 'classifier__estimator__n_estimators': 20,
 'count_vectorizer__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [4]:
class SentenceVectorizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.transformer = SentenceTransformer('all-MiniLM-L6-v2')
        return self

    def transform(self, X):
        embeddings = self.transformer.encode(list(X))
        return embeddings
    
pipeline = Pipeline([
    ('features', FeatureUnion([
        # TFIDF Features
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        # Tag Count Features
        ('tag_pipeline', Pipeline([
            ('count_tag', CountVectorizer(tokenizer=get_tag)),
            ('tag_tfidf', TfidfTransformer())
        ])),
    ])),

    ('classifier', MultiOutputClassifier(RandomForestClassifier())),
])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2), (1, 3)),
    'features__tag_pipeline__count_tag__ngram_range': ((1, 1), (1, 2), (1, 3), (1, 4), (1, 5)),
    'classifier__estimator__n_estimators': [2,5,10,20,50,100],
    'classifier__estimator__min_samples_split': [2,5,10,20,30]
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train,Y_train)
cv.best_params_

In [ ]:
pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)

In [417]:
output_category = list(df.iloc[:,3:].columns)
Y_pred_train = pipeline.predict(X_train)
for i in range(Y_train.shape[1]):
    print(output_category[i])
    print(classification_report(Y_train.T[i], Y_pred_train.T[i]))

related
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      4577
           1       0.99      1.00      0.99     15083

    accuracy                           0.99     19660
   macro avg       0.99      0.97      0.98     19660
weighted avg       0.99      0.99      0.99     19660

request
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     16276
           1       0.99      0.95      0.97      3384

    accuracy                           0.99     19660
   macro avg       0.99      0.98      0.98     19660
weighted avg       0.99      0.99      0.99     19660

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19575
           1       1.00      0.65      0.79        85

    accuracy                           1.00     19660
   macro avg       1.00      0.82      0.89     19660
weighted avg       1.00      1.00      1.00     1966

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17830
           1       1.00      0.96      0.98      1830

    accuracy                           1.00     19660
   macro avg       1.00      0.98      0.99     19660
weighted avg       1.00      1.00      1.00     19660

fire
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19454
           1       1.00      0.75      0.86       206

    accuracy                           1.00     19660
   macro avg       1.00      0.87      0.93     19660
weighted avg       1.00      1.00      1.00     19660

earthquake
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17821
           1       0.98      0.97      0.98      1839

    accuracy                           1.00     19660
   macro avg       0.99      0.98      0.99     19660
weighted avg       1.00      1.00      1.00     19660

col

In [435]:
df_sample = X_test[Y_test[:,0] != Y_pred[:,0]]
for text in df_sample:
    print(text)
    print()

I want to sign up for the 4636 program, please. 

ISAF helicopter crews flew reconnaissance missions and transported equipment to be used by ANSF to assist villagers.

I wrote twice already and no answer. we are waiting please 

PLEASE GIVE ME A DIGICEL PHONE CARD SO I CAN CALL SOMEONE. 

my soul cry for you lord, my soul cry for that I liberate any problem, my soul cry for itself liberate 

NOTES: Whether forecast. I have heard it mentioned by other translators previously.

Recently EPAC assisted a group of citizens from Brichmulla, a town 150 kilometers from Tashkent and the site of Islamic Movement of Uzbekistan incursions in mid-2000, with a project to plant safflower seeds in 50 hectares of exposed land along the Chervok Water Reserve.

The cyclone that has anonncer, is that it will rain 

In the last six months, we injected 7,000 tons of improved-seed wheat varieties that we collected all over Central Asia that is drought resistant.

A deficient monsoon this year could significan

In [418]:
output_category = list(df.iloc[:,3:].columns)
for i in range(Y_test.shape[1]):
    print(output_category[i])
    print(classification_report(Y_test.T[i], Y_pred.T[i]))

related
              precision    recall  f1-score   support

           0       0.64      0.42      0.51      1544
           1       0.84      0.93      0.88      5010

    accuracy                           0.81      6554
   macro avg       0.74      0.68      0.70      6554
weighted avg       0.79      0.81      0.79      6554

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5464
           1       0.83      0.50      0.62      1090

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6478
           1       0.00      0.00      0.00        76

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5938
           1       0.91      0.79      0.85       616

    accuracy                           0.97      6554
   macro avg       0.94      0.89      0.92      6554
weighted avg       0.97      0.97      0.97      6554

cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6416
           1       0.71      0.04      0.07       138

    accuracy                           0.98      6554
   macro avg       0.85      0.52      0.53      6554
weighted avg       0.97      0.98      0.97      6554

oth

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.